This project aims at analyzing movielens data by using the following 3 methods: Collaborative Filtering(KNN),  Apriori, Matrix Factorization.

In [205]:
#load neccessary files and packages
import pandas as pd
data=pd.read_csv("/content/ratings.csv")
movieinfo = pd.read_csv('/content/movies.csv')
!pip install deepctr
!pip install surprise

In [210]:
# Explore the sparcity
print(data.shape[0]/data['userId'].unique().shape[0]/data['movieId'].unique().shape[0])
print(data)

0.016999683055613623
        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]


In [76]:
from surprise import KNNWithMeans
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import KFold

# Data loading
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data_SVD = Dataset.load_from_file('/content/ratings.csv', reader=reader)
SVD_train_set = data_SVD.build_full_trainset()

# KNN algorithm
KNNalgo = KNNWithMeans(k=50, sim_options={'user_based': False, 'verbose': 'True'})
# K-fold method
kf = KFold(n_splits=3)
# Train model
for trainset, testset in kf.split(data_SVD):
  KNNalgo.fit(trainset)
  predictions = KNNalgo.test(testset)
  # Calculating RMSE
  accuracy.rmse(predictions, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8964
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9077
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8950


In [132]:
#Input userid and movie id that you want to 
uid = 1
iid = 6
# Check the difference between real rating and predicted rating
KNNpred = KNNalgo.predict(str(uid), str(iid))
print(data[(data['userId']==uid)&(data['movieId']==iid)].drop(['timestamp'],axis=1))
print('Pred Rating:',KNNpred.est)

   userId  movieId  rating
2       1        6     4.0
Pred Rating: 4.697345152783779


In [200]:
userid_list=data['userId'].unique()
movieid_list=data['movieId'].unique()
print(userid_list.shape[0]*mivieid_list.shape[0])
KNNtable=pd.DataFrame()
# Input the userid that you want to predict for all movies
i=5 #Number 5 user
for j in range(0,mivieid_list.shape[0]-1):
  # If can not find the real rating, then mark null as haven't watched
  try: realdata=data[(data['userId']==userid_list[i])&(data['movieId']==movieid_list[j])]['rating'].iloc[0]
  except: realdata='null'
  KNNtable=KNNtable.append({'user_id': userid_list[i],'movie_id':movieid_list[j],'predict_rating':KNNalgo.predict(str(userid_list[i]), str(movieid_list[j])).est,'real_rating':realdata}, ignore_index=True)

5931640


In [201]:
# Print prediction result to all movies and output as file "PredictionTable.csv""
KNNtable = KNNtable.merge(movieinfo,how='left',left_on='movie_id',right_on='movieId',)
pd.set_option('display.max_rows', 50)
print(KNNtable.sort_values(by='predict_rating',ascending=False))
KNNtable.to_csv('./PredictionTable.csv', sep=',', header=True, index=True)

      movie_id  predict_rating real_rating  user_id  movieId  \
3975  141928.0             5.0        null      6.0   141928   
3991  149350.0             5.0        null      6.0   149350   
1985  110130.0             5.0        null      6.0   110130   
3999  156025.0             5.0        null      6.0   156025   
7168   95149.0             5.0        null      6.0    95149   
...        ...             ...         ...      ...      ...   
8852  109897.0             1.0        null      6.0   109897   
8855  114396.0             1.0        null      6.0   114396   
8859  122246.0             1.0        null      6.0   122246   
8862  134246.0             1.0        null      6.0   134246   
857     5962.0             1.0        null      6.0     5962   

                                       title  \
3975            Bloodsucking Bastards (2015)   
3991                   Lumberjack Man (2015)   
1985                     Nut Job, The (2014)   
3999   Ice Age: The Great Egg-Scapade (

In [118]:
from surprise import SVD
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
import pandas as pd


# ALS optimization
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD
#bsl_options = {'method': 'sgd','n_epochs': 5}
BSLalgo = BaselineOnly(bsl_options=bsl_options)
#algo = BaselineOnly()
#algo = NormalPredictor()

# K-fold
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data_SVD):
    # Train
    BSLalgo.fit(trainset)
    predictions = BSLalgo.test(testset)
    # RMSE
    accuracy.rmse(predictions, verbose=True)
    accuracy.

Estimating biases using als...
RMSE: 0.8674
Estimating biases using als...
RMSE: 0.8693
Estimating biases using als...
RMSE: 0.8750


In [203]:
uid = 610
iid = 170875

BSLpred = BSLalgo.predict(str(uid), str(iid))
print(data[(data['userId']==uid)&(data['movieId']==iid)].drop(['timestamp'],axis=1))
print('Pred Rating:',BSLpred.est)

        userId  movieId  rating
100835     610   170875     3.0
Pred Rating: 3.440885611164198


In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import NFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names
sparse_features = ["userId", "movieId", 'timestamp']
target = ['rating']

# Label Encoding
FMdata = data
for feature in sparse_features:
    lbe = LabelEncoder()
    FMdata[feature] = lbe.fit_transform(FMdata[feature])
# Calculate the # of Features
fixlen_feature_columns = [SparseFeat(feature, FMdata[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# Data Split
train, test = train_test_split(FMdata, test_size=0.3)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# Train with NFM
model = NFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.3 )
# Pred
pred_ans = model.predict(test_model_input, batch_size=256)
# RMSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


194/194 [==============================] - 3s 15ms/step - loss: 3.1341 - mse: 3.1340 - val_loss: 0.9740 - val_mse: 0.9736
test RMSE 0.9925220400575495


In [10]:
all_input={name:data[name].values for name in feature_names}
all_ans = model.predict(all_input,batch_size=256)

In [12]:
data=pd.read_csv("/content/ratings.csv")
pred_matrix=data
pred_matrix['pred']=all_ans.flatten()
pred_matrix=pred_matrix.drop(['timestamp'],axis=1)

uid = 610
iid = 170875
print(pred_matrix[(pred_matrix['userId']==uid)&(pred_matrix['movieId']==iid)])


        userId  movieId  rating      pred
100835     610   170875     3.0  3.265137
